# Part 2
For part 2 of the project, you will be engineering a MySQL database for your IMDB movie data. You will examine the data to determine the correct SQL data type and size for each column. You will then construct the ERD and forward engineer the database. Finally, you will insert your IMDB data into your database from a jupyter notebook.

## Specifications - Database
Your stakeholder wants you to take your data from Part 1 of the project and to create a MySQL database with the requested columns/features.
Specifically, they would like the following tables and columns included in your database:

A "title_basics" table, which contains:
- tconst (as the primary key)
- primary_title
- start_year
- runtime

A "ratings" table, which contains::
- tconst (as the primary key)
- average_rating
- number_of_votes

They want the Genres from title basics included in the database, but they want the tables normalized. You will learn how to process and normalize the genres next week. For now, you will include the tables in your database, but will not insert any data until Part 3.

A "genres" table, which includes:
- genre_id (New): a numeric ID for each genre (primary key)
- genre_name (New): the full name of each genre

A "title_genres" table (or "title_has_genres" table), which includes:
- tconst
- genre_id

Hint: let MySQL's ERD editor help you create this table.

In [1]:
import pandas as pd
import json, os

In [2]:
os.listdir("Data/")

['title-ratings.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'title-akas-us-only.csv',
 '.ipynb_checkpoints',
 'title-basics.csv']

In [12]:
# Load in data for basics
basics = pd.read_csv('Data/title-basics.csv')
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86979 entries, 0 to 86978
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  int64  
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  86979 non-null  int64  
 8   genres          86979 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 6.0+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [13]:
# Load in data for ratings
ratings = pd.read_csv('Data/title-ratings.csv')
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71900 entries, 0 to 71899
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


,tconst,averageRating,numVotes
0,tt0035423,6.4,87153
1,tt0062336,6.4,175
2,tt0069049,6.7,7754
3,tt0088751,5.2,336
4,tt0096056,5.6,846


In [7]:
def string_lengths(df):
    print("\nSTRING COLUMN MAX LENGTHS:")
    for col in df.select_dtypes('object').columns:
        length =  df[col].fillna('').map(len).max()
        print(f"- {col}: {length}")

In [9]:
# Show column lengths for basics df
string_lengths(basics)


STRING COLUMN MAX LENGTHS:
- tconst: 10
- titleType: 5
- primaryTitle: 242
- originalTitle: 242
- genres: 29


In [10]:
# Show column lengths for ratings df
string_lengths(ratings)


STRING COLUMN MAX LENGTHS:
- tconst: 10


## Design the Database Model/ERD

## Create the Database with your CREATE SQL script

## Confirm the Database has been updated correctly